## PDF Document Loaders
- Load various kind of documents from the web and local files.
- Apply LLM to the documents for summarization and question answering.

### Project 1: Question Answering from PDF Document
- We will load the document from the local file and apply LLM to answer the questions.
- Lets use research paper published on the missuse of the health supplements for workout. 

In [ ]:

# !pip install pymupdf tiktoken


In [ ]:
from dotenv import load_dotenv

load_dotenv('./../.env')

True

In [3]:
from langchain_community.document_loaders import PyMuPDFLoader

loader = PyMuPDFLoader("rag-dataset/health supplements/1. dietary supplements - for whom.pdf")

docs = loader.load()

In [4]:
len(docs)

17

In [5]:
# docs[0].metadata
# print(docs[0].page_content)

In [6]:
### Read the list of PDFs in the dir
import os

pdfs = []
for root, dirs, files in os.walk("rag-dataset"):
    # print(root, dirs, files)
    for file in files:
        if file.endswith(".pdf"):
            pdfs.append(os.path.join(root, file))

In [7]:
docs = []
for pdf in pdfs:
    loader = PyMuPDFLoader(pdf)
    temp = loader.load()
    docs.extend(temp)

    # print(temp)
    # break

In [8]:
len(docs)

64

In [9]:
def format_docs(docs):
    return "\n\n".join([x.page_content for x in docs])


context = format_docs(docs)

In [10]:
docs[0]

Document(metadata={'producer': 'iLovePDF', 'creator': '', 'creationdate': '', 'source': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'file_path': 'rag-dataset\\gym supplements\\1. Analysis of Actual Fitness Supplement.pdf', 'total_pages': 15, 'format': 'PDF 1.7', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2024-10-21T11:38:50+00:00', 'trapped': '', 'modDate': 'D:20241021113850Z', 'creationDate': '', 'page': 0}, page_content='Citation: Espeño, P.R.; Ong, A.K.S.;\nGerman, J.D.; Gumasing, M.J.J.; Casas,\nE.S. Analysis of Actual Fitness\nSupplement Consumption among\nHealth and Fitness Enthusiasts. Foods\n2024, 13, 1424. https://doi.org/\n10.3390/foods13091424\nAcademic Editors: Ilija Djekic\nand Nada Smigic\nReceived: 30 March 2024\nRevised: 15 April 2024\nAccepted: 18 April 2024\nPublished: 6 May 2024\nCopyright: © 2024 by the authors.\nLicensee MDPI, Basel, Switzerland.\nThis article is an open access article\ndistributed\nunder

In [11]:
# print(context)

In [12]:
import tiktoken

encoding = tiktoken.encoding_for_model("gpt-4o-mini")


In [13]:
encoding.encode("congratulations"), encoding.encode("rqsqeft")

([542, 111291, 14571], [81, 31847, 80, 5276])

In [14]:
len(encoding.encode(docs[0].page_content))

968

In [15]:
len(encoding.encode(context))

60268

In [16]:
969*64

62016

In [17]:
### Question Answering using LLM
from langchain_ollama import ChatOllama

from langchain_core.prompts import (SystemMessagePromptTemplate, HumanMessagePromptTemplate,
                                    ChatPromptTemplate)



from langchain_core.output_parsers import StrOutputParser

base_url = "http://localhost:11434"
model = 'llama3.2:3b'

llm = ChatOllama(base_url=base_url, model=model)


In [18]:
system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who answer user question based on the provided context. 
                                                    Do not answer in more than {words} words""")

prompt = """Answer user question based on the provided context ONLY! If you do not know the answer, just say "I don't know".
            ### Context:
            {context}

            ### Question:
            {question}

            ### Answer:"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

# template
# template.invoke({'context': context, 'question': "How to gain muscle mass?", 'words': 50})

qna_chain = template | llm | StrOutputParser()

In [19]:
qna_chain

ChatPromptTemplate(input_variables=['context', 'question', 'words'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['words'], input_types={}, partial_variables={}, template='You are helpful AI assistant who answer user question based on the provided context. \n                                                    Do not answer in more than {words} words'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, template='Answer user question based on the provided context ONLY! If you do not know the answer, just say "I don\'t know".\n            ### Context:\n            {context}\n\n            ### Question:\n            {question}\n\n            ### Answer:'), additional_kwargs={})])
| ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
| StrOutputParser()

In [20]:
response = qna_chain.invoke({'context': context, 'question': "How to gain muscle mass?", 'words': 50})
print(response)

It seems like you provided a random answer instead of responding to the question about how to gain muscle mass.

To answer your original question, gaining muscle mass requires a combination of proper nutrition, consistent training, and sufficient rest. Here are some tips:

1. **Eat enough protein**: Protein is essential for building and repairing muscle tissue. Aim for 1.2-1.6 grams of protein per kilogram of body weight daily.
2. **Resistance training**: Focus on weightlifting exercises that work multiple muscle groups at once, such as squats, deadlifts, bench press, and rows.
3. **Progressive overload**: Gradually increase the weight or resistance you're lifting over time to challenge your muscles and stimulate growth.
4. **Rest and recovery**: Adequate rest and recovery are crucial for muscle growth. Ensure you're getting 7-9 hours of sleep per night and taking rest days as needed.
5. **Caloric surplus**: Consume more calories than you burn, with a focus on carbohydrates and protein

In [ ]:
response = qna_chain.invoke({'context': context, 'question': "How to reduce the weight?", 'words': 50})
print(response)

In [ ]:
response = qna_chain.invoke({'context': context, 'question': "How to do weight loss?", 'words': 50})
print(response)

The question is "How to do weight loss?" and the answer seems to be a distraction or an unrelated statement, as it says "Compared to the above outcomes, more is known about potential herb-drug interactions. ... How to do weight loss?". However, I'll try to provide a response based on the context.

To do weight loss, consider the following general tips:

1. **Maintain a balanced diet**: Focus on whole, unprocessed foods like vegetables, fruits, lean proteins, and whole grains.
2. **Stay hydrated**: Drink plenty of water throughout the day.
3. **Exercise regularly**: Aim for at least 150 minutes of moderate-intensity aerobic exercise or 75 minutes of vigorous-intensity aerobic exercise per week.
4. **Get enough sleep**: Aim for 7-9 hours of sleep per night to help regulate hunger hormones and support weight loss.
5. **Be mindful of portion sizes**: Pay attention to the serving sizes of your food and try to eat until you're satisfied, not stuffed.
6. **Keep track of your progress**: Use a

In [ ]:
response = qna_chain.invoke({'context': context, 'question': "How many planets are there outside of our solar system?", 'words': 50})
print(response)

It looks like you've provided a random answer, but it's not related to the text. The correct question and answer aren't even present in your response.

However, I can help you with the original text. If you'd like to ask a specific question about the content of the text, I'll do my best to provide an accurate answer.


### Project 2: PDF Document Summarization

In [57]:
system = SystemMessagePromptTemplate.from_template("""You are helpful AI assistant who works as document summarizer. 
                                                   You must not hallucinate or provide any false information.""")

prompt = """Summarize the given context in {words}.
            ### Context:
            {context}

            ### Summary:"""

prompt = HumanMessagePromptTemplate.from_template(prompt)

messages = [system, prompt]
template = ChatPromptTemplate(messages)

summary_chain = template | llm | StrOutputParser()

In [58]:
summary_chain

ChatPromptTemplate(input_variables=['context', 'words'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are helpful AI assistant who works as document summarizer. \n                                                   You must not hallucinate or provide any false information.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'words'], input_types={}, partial_variables={}, template='Summarize the given context in {words}.\n            ### Context:\n            {context}\n\n            ### Summary:'), additional_kwargs={})])
| ChatOllama(model='llama3.2:3b', base_url='http://localhost:11434')
| StrOutputParser()

In [59]:
response = summary_chain.invoke({'context': context, 'words': 50})
print(response)

This article discusses the potential therapeutic and toxic mechanisms of action of botanical supplements, also known as herbal remedies. The authors review various botanicals, their primary active constituents, typical use, dosage, and reported adverse effects. They note that concurrent exposure to other compounds and the heterogeneity of herbal supplements can make it difficult to determine the specific mechanism of toxicity. Case reports have identified potential mechanisms for liver toxicity in some botanicals, such as black cohosh, kava kava, saw palmetto, and milk thistle. Other botanicals, such as yohimbe and ginkgo biloba, have been associated with non-hepatic symptoms like seizures and bleeding. The article also discusses potential herb-drug interactions, where the activation of metabolizing enzymes can affect the pharmacokinetics of drugs.

Some key points from the article include:

* Botanical supplements can have therapeutic and toxic mechanisms of action.
* Many botanicals 

In [61]:
response = summary_chain.invoke({'context': context, 'words': 500})
print(response)

This article discusses the potential toxicities and interactions of botanical supplements, including their active compounds, typical use, dosage, and reported adverse effects. The authors review various case reports and in vivo studies to highlight the mechanisms underlying these adverse effects, such as mitochondrial dysfunction, oxidative stress, and alteration of bile acid homeostasis. They also discuss herb-drug interactions, including induction or suppression of metabolizing enzymes, which can affect the pharmacokinetics of drugs. The article emphasizes the need for caution when using botanical supplements, particularly in combination with other compounds, and highlights the importance of monitoring patients for potential adverse effects.


### Project 3: Report Generation from PDF Document

Streamlit Tutorial: https://www.youtube.com/watch?v=hff2tHUzxJM&list=PLc2rvfiptPSSpZ99EnJbH5LjTJ_nOoSWW

In [62]:
response = qna_chain.invoke({'context': context, 
                             'question': "Provide a detailed report from the provided context. Write answer in Markdown.", 
                             'words': 2000})
print(response)

**Botanical Supplements and Potential Adverse Effects**

The use of botanical supplements has become increasingly popular, with many people turning to these products for their purported health benefits. However, like any other substance, botanicals can also pose potential risks and adverse effects.

**Commonly Used Botanical Supplements and Their Potential Risks**

1. **Black Cohosh (Cimicifuga racemosa)**
	* Associated with jaundice and liver failure in menopausal women
	* Possible mechanisms: mitochondrial dysfunction, oxidative stress, and alteration of bile acid homeostasis
2. **Kava Kava**
	* Linked to liver toxicity, sometimes requiring transplants
	* Possible mechanisms: depletion of glutathione, increasing oxidative stress, and inhibition of cyclooxygenases (mitochondrial dysfunction)
3. **Saw Palmetto**
	* Use has been associated with cholestatic hepatitis and pancreatitis
	* Possible mechanism: alterations in bile secretion
4. **Echinacea**
	* Cholestatic symptoms were seen i